In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
from PIL import Image
import shutil
import xlsxwriter as xw
import requests
import urllib.request
import time
import sys
import re
import os


#--------------------- 경로 지정 ----------------------------------------
now = time.strftime('%y-%m-%d_%H-%M-%S') #현재시간

driver = webdriver.Chrome('C:\Temp\chromedriver_win32\chromedriver.exe') #크롬드라이버 경로

print('--------------------------------------------------------------')
print('G 마켓의 분야별 Best Seller 상품 정보 추출하기')
print('==============================================================')
print('')
print('')
print('')

save_path = 'C:/test/14week3/' # 저장 경로
os.makedirs(save_path + now + '-' + 'G마켓')
save_path = (save_path + now + '-' + 'G마켓/')

driver.get('http://corners.gmarket.co.kr/Bestsellers')   #타겟 주소
time.sleep(2)

#--------------------- 문서 스크롤 (이미지, 상품 로딩) 후 크롤링 시작 ----------------------------------------

page_length = driver.execute_script("return document.body.scrollHeight") 
count = 0
while count < page_length:
    count +=900
    driver.execute_script('window.scrollTo(0, '+str(count)+')')
    print('상품 이미지를 로드 중 입니다. 기다려 주세요.'+str(count/page_length*100)+"%진행")
    time.sleep(1)
    
soup =  BeautifulSoup(driver.page_source, 'html.parser')

f = open(save_path + 'G마켓 베스트 상품.txt', 'a', encoding='utf-8') #쓰기 모드

os.makedirs(save_path + 'image/')

seller_rank = []
seller_name = []
seller_img = []
seller_price = []
seller_dis_price = []
seller_dis_rate = []

for i in range(0, 50): #건수 설정 200을 초과 할 수 없음
    
    seller_rank_temp = soup.select('#no'+ str(i+1))[0].text   #순위                                
    seller_img_temp = soup.select('#gBestWrap > div > div:nth-child(5) > div:nth-child(3) > ul > li:nth-child('+ str(i+1) +') > div.thumb > a')
    seller_img_temp = seller_img_temp[0].find('img')["src"] #이미지 주소 추출
    urllib.request.urlretrieve(seller_img_temp, save_path + 'image/' + str(i+1) + '.jpg')   #이미지 다운로드                   
    
    seller_name_temp = soup.select('#gBestWrap > div > div:nth-child(5) > div:nth-child(3) > ul > li:nth-child('+ str(i+1) +') > a')[0].text   #상품명
#--------------------- 원가만 있을 때 원가, 최저가, 할인율 예외처리 ------------------------------------------------
    try:
        seller_price_temp = soup.select('#gBestWrap > div > div:nth-child(5) > div:nth-child(3) > ul > li:nth-child('+ str(i+1) +') > div.item_price > div.o-price > span > span')[0].text   #원가
    except: #원가만 있을 때
        seller_price_temp = soup.select('#gBestWrap > div > div:nth-child(5) > div:nth-child(3) > ul > li:nth-child('+ str(i+1) +') > div.item_price > div.s-price > strong > span > span')[0].text 
        
    seller_dis_price_temp = soup.select('#gBestWrap > div > div:nth-child(5) > div:nth-child(3) > ul > li:nth-child('+ str(i+1) +') > div.item_price > div.s-price > strong > span > span')[0].text   #할인가

    try: 
        seller_dis_rate_temp = soup.select('#gBestWrap > div > div:nth-child(5) > div:nth-child(3) > ul > li:nth-child('+ str(i+1) +') > div.item_price > div.s-price > span > em')[0].text   #할인율

    except: # 할인율 찾을 수 없을 때
        seller_dis_rate_temp = '할인율 없음.'

    print("-----------------------------------------------------------------------------------------------")
    print("판매 순위: " + seller_rank_temp)
    print("상품명: " + seller_name_temp)
    print("원가: " + seller_price_temp)
    print("최저가: " + seller_dis_price_temp)
    print("할인율: " + seller_dis_rate_temp)
    print("")

    f.write("-----------------------------------------------------------------------------------------------")
    f.write("판매 순위: "+ seller_rank_temp + "\n")
    f.write("상품명: "+ seller_name_temp + "\n")
    f.write("원가: "+ seller_price_temp + "\n")
    f.write("최저가: "+ seller_dis_price_temp + "\n")
    f.write("할인율: "+ seller_dis_rate_temp + "\n")
    f.write("\n")

    seller_rank.append(seller_rank_temp)
    seller_name.append(seller_name_temp)
    seller_img.append(seller_img_temp)
    seller_price.append(seller_price_temp)
    seller_dis_price.append(seller_dis_price_temp)
    seller_dis_rate.append(seller_dis_rate_temp)
    
f.close()

#--------------------- xls, csv 파일 생성 ------------------------------------------------
wb = xw.Workbook(save_path + 'G마켓 베스트 상품.xls')
ws = wb.add_worksheet()

ws.set_default_row(80) # 행 크기

ws.write(0,0, "판매 순위")
ws.write(0,1, "이미지")
ws.write(0,2, "제품명")
ws.write(0,3, "원가")                        
ws.write(0,4, "최저가")
ws.write(0,5, "할인율")

for i in range(0,50): #위의 건수와 통일
    ws.set_column(0, i+1, 30) # 열 크기
    
    image = Image.open(save_path + 'image/'+str(i+1)+'.jpg') # 이미지 로드
    resize_image = image.resize((100,100)) # 이미지 크기조정 
    resize_image.save(save_path + 'image/'+str(i+1)+'.jpg') # 이미지 저장

    ws.write(i+1, 0, seller_rank[i])
    ws.insert_image(i+1, 1, save_path + 'image/'+str(i+1)+'.jpg')
    ws.write(i+1, 2, seller_name[i])
    ws.write(i+1, 3, seller_price[i])
    ws.write(i+1, 4, seller_dis_price[i])
    ws.write(i+1, 5, seller_dis_rate[i])

wb.close()

shutil.copyfile(save_path+'G마켓 베스트 상품.xls', save_path+"G마켓 베스트 상품.csv") # 파일 생성
driver.close( )
print('크롤링 종료')
os.system("pause")

--------------------------------------------------------------
G 마켓의 분야별 Best Seller 상품 정보 추출하기



상품 이미지를 로드 중 입니다. 기다려 주세요.4.328171587958065%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.8.65634317591613%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.12.984514763874195%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.17.31268635183226%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.21.640857939790322%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.25.96902952774839%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.30.29720111570645%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.34.62537270366452%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.38.953544291622585%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.43.281715879580645%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.47.60988746753871%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.51.93805905549678%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.56.26623064345484%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.60.5944022314129%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.64.92257381937098%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.69.25074540732903%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.73.5789169952871%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.77.90708858324517%진행
상품 이미지를 로드 중 입니다. 기다려 주세요.82.23526017120322%진행
상품 이미지를

-----------------------------------------------------------------------------------------------
판매 순위: 40
상품명: [송월타월]애니멀즈 고리수건 5장 +사은품/동물자수/어린이집
원가: 19,000원
최저가: 8,900원
할인율: 53%

-----------------------------------------------------------------------------------------------
판매 순위: 41
상품명: [아디다스](신세계의정부점)[New Performance] QUESTAR FLOW NXT M_[FY5951 FY9559]
원가: 53,400원
최저가: 36,520원
할인율: 31%

-----------------------------------------------------------------------------------------------
판매 순위: 42
상품명: [설화수](신세계강남점)[6월]순행클렌징폼
원가: 37,000원
최저가: 31,450원
할인율: 15%

-----------------------------------------------------------------------------------------------
판매 순위: 43
상품명: 31건어물 순살 아귀포 (중) 10장
원가: 13,800원
최저가: 6,900원
할인율: 50%

-----------------------------------------------------------------------------------------------
판매 순위: 44
상품명: 호두잉글리시 M 6개월 이용권
원가: 118,000원
최저가: 59,000원
할인율: 50%

-----------------------------------------------------------------------------------------------
판매 순위: 45
상

0